In [ ]:
import numpy as np
import cv2 as cv
import os
import tensorflow as tf

Lab code to unzip the database:

In [ ]:
####    GIVEN CODE    ####
import zipfile
!rm /content/download
!rm -r /content/imagedb/
!rm -r /content/imagedb_test
!wget https://vc.ee.duth.gr:6960/index.php/s/wlnkxtlGmqBeATC/download
local_zip = '/content/download'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
# Declaring training and testing directories 
train_dir = '/content/imagedb'                # train_folders
test_dir  = '/content/imagedb_test'           # testing_folders

In [ ]:
nodes = 32 
epochs = 600     
do = 0.5  # drop out rate
bs = 20   # batch_size
ks = 3    # kernel size

image_dimension = 256

In [ ]:
## Auto find the nodes of the last layer ##
classes = []
for dirname, _, filenames in os.walk('/content/imagedb_test'):
  # print(dirname)
  a,folder = dirname.split("/content/imagedb_test")
  folder = folder[1:]
  # print(folder)
  classes.append(folder)

classes = classes[1:]
print(classes)

out_nodes = len(classes)
print("\nThe nodes of the last layer and the number of classes is: " + str(out_nodes))

In [ ]:
# Building the model
from keras import models
from keras import layers
from keras import optimizers

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

# Load the pre-trained model
pre_trained = ResNet50(include_top=False, #whether we are including a fully connected layer or not
                       weights='imagenet',
                       input_shape=(224, 224, 3))
# pre_trained.summary()

In [ ]:
# # Keeping active the first 100 layers
# for layer in pre_trained.layers[:]:
#     layer.trainable = False

# # Check the trainable status of the individual layers
# for layer in pre_trained.layers:
#     print(layer, layer.trainable)

In [ ]:
# Connecting the pre-trained model with my model
model = models.Sequential()
model.add(pre_trained)

    # nodes = 32
model.add(layers.Conv2D(filters=nodes*4, kernel_size=(ks,ks), activation='relu'))
model.add(layers.Conv2D(filters=nodes*4, kernel_size=(ks,ks), activation='relu'))
model.add(layers.Conv2D(filters=nodes*4, kernel_size=(ks,ks), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(3,3), padding='same'))

model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(rate=do))

model.add(layers.Dense(out_nodes,activation='softmax'))


model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

# model.summary()

In [ ]:
from keras.applications import ResNet50
# DATA AGMENTATION 
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale = 1./255, 
                                    rotation_range = 10,
                                    brightness_range = [0.5 , 1],
                                    zoom_range = [0.15 , 1],
                                    validation_split = 0.2,
                                    horizontal_flip=False,
                                    vertical_flip=False,
                                    fill_mode="nearest")

# --------------------
# Flow training images in batches of bs using train_data_gen generator
# --------------------
train_generator = train_data_gen.flow_from_directory(train_dir,
                                                    batch_size=bs,
                                                    class_mode='categorical',
                                                    # color_mode='grayscale',
                                                    target_size=(image_dimension, image_dimension),
                                                    shuffle=True,
                                                    subset='training', seed=1)     
# --------------------
# Flow validation images in batches of bs using train_data_gen generator
# --------------------
validation_generator = train_data_gen.flow_from_directory(train_dir,
                                                      batch_size=100,
                                                      class_mode='categorical',
                                                      # color_mode='grayscale',
                                                      target_size=(image_dimension, image_dimension),
                                                      subset='validation', seed=1) 

In [ ]:
# ----- Train the model -----

callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'hw4.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                       min_delta=0, 
                                                       patience=5, 
                                                       verbose=1, 
                                                       mode='auto', 
                                                       restore_best_weights=True)
callbacks.append(early_stop_callback)


history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=epochs,
      validation_data=validation_generator,
      verbose=1,
      shuffle=True,
      callbacks=callbacks)

In [ ]:
# --------------------
# Flow validation images in batches of 20 using datagen generator
# --------------------
test_data_gen = ImageDataGenerator( rescale = 1./255, 
                                    brightness_range = [0.5 , 1],
                                    zoom_range = [0.15 , 1],
                                    horizontal_flip=False,
                                    vertical_flip=False
                                   )

test_generator = test_data_gen.flow_from_directory(test_dir,
                                                   batch_size=100,
                                                   class_mode='categorical',
                                                   # color_mode='grayscale',
                                                   shuffle=False,
                                                   target_size=(image_dimension, image_dimension)) 

test_loss, test_acc = model.evaluate(test_generator)